###  This is the master company addition file.
####  It adds entries to the nedbank_companies collection in lookubs db on the MongoDb

```
Load some libraries:
```

In [1]:
import sys
sys.path.insert(0, '/home/christo/notebooks/christo/prod/')
sys.path.insert(0, '/home/christo/notebooks/christo/company_maintenance_scripts/')
from nedbank_companies_addition_lib import maxindexfn, mongo_regex_searchstring
from nedbank_companies_addition_lib import nedbankcompany_class_check, classlookupfn, input_dict_fn, mongo_regex_searchstring_row
from nedbank_companies_addition_lib import  nedbank_companies_db, regex_searchstring

import re
import pandas as pd
from NedbankSpendPy import  client
import time
import numpy as np
from collections import Counter
from datetime import datetime

Assign the subclass_id.  Check if it exists.  If it doesn't, it needs to be added using new_classification.js
Find the appropriate subclass_id in "Classifications Standard Industrial Classification of all Economic Activities
(SIC) Seventh Edition" here (http://www.statssa.gov.za/classifications/codelists/Web_SIC7a/SIC_7_Final_Manual_Errata.pdf)

#### The following creates generic avoid list.  It draws the aliases from all entries of a list of entries.

In [2]:
nedbank_companies_df=pd.DataFrame(list(client.lookups['nedbank_companies'].find({})))
subclass_id_list=[56101,47110,47610,45500,47620]
generic_avoidlist=[]
for subclass_id in subclass_id_list:
    for aliases in nedbank_companies_df[nedbank_companies_df.subclass_id==subclass_id].aliases:
        for alias in aliases:
#             print(alias)
            generic_avoidlist.append(alias)

In [3]:
generic_avoidlist.append('NANDOS')
generic_avoidlist.index('NANDOS')
generic_avoidlist.append('OCEAN BASKET')
generic_avoidlist.append('BEAUTY')
generic_avoidlist.append('SERVICE STATION')
generic_avoidlist.append('KauaiCascades')
generic_avoidlist.append('WimpyCabanas')
generic_avoidlist.append('QUEENSPARK')
generic_avoidlist.append('DALPARK PLAZA')
generic_avoidlist.append('BAR')
generic_avoidlist.append('RESTA')
# DALPARK PLAZA
generic_avoidlist=list(set(generic_avoidlist))

In [4]:
subclass_id = 55101 # look carefully at this assignment!
companyname='TRAVELSTART'; # Assign the companyname.  This how it is found by other scripts!
groupname='TRAVEL'; # Assign the group name:
aliases=['TRAVELSTART'];# Define the alias list. This is IMPORTANT. 
# It is the key code that will find this entity in the clientswipes db 
# It is the actual string that will search the franchise names
generic_avoidlist.append('NANDOS');# Define the anti-aliases, if NO antialias is necessary leave the list EMPTY, NOT and empty string:
channel = 0, # bricks and mortar == 0; mixed == 1; online == 2
discretionary = 2; #non-discretionary == 0; mixed == 1; discretionary == 2
period='201907'
start_date=datetime(2019,7,1)
end_date=datetime(2019,7,31)
generic_avoidlist=list(set(generic_avoidlist))

Define aliases and anti-aliases.  Aliases define strings that should also be found.  Anti aliases are those substrings 

In [31]:
aliases=['TAXIFYGLOBA_',
 'PAYU*TAXIFY.EU',
 '. sz. bolt -',
 'PAYPAELLE BOLT',
#  '/BOLT/DEN',
 'IKH*Taxify',
 'TAXIFY SOUTH AFRICA',
#  '/BOLT/PET',
 'TAXIFY ..',
 'TAXIFY SOUTH AFRICA',
#  'BOLT.EU /R/',
 'Lesedi Taxify',
 'Bolt',
 'POS BOLT -   INS FUNDS',
 'Taxify -',
 'Bolt -  -',
#  '/BOLT/MAX',
 'LIDL . sz. bolt',
 'PAYPAL *BOLT T',
 'TAXIFY _',
#  '/BOLT/INASP',
#  '/BOLT/PETSP',
 'TAXIFY SOUTH A',
 'Taxify',
 'www.taxify.co.z',
#  '/BOLT/ALE',
 'TAXIFY_PAYU',
 'bolt.eu',
 'Taxify _',
#  '/BOLT/LAU',
 'Taxify --',
 'TAXIFY MAHLANG',
 'BOLT.EU',
 'TAXIFYGLOBA.. :',
 'POS BOLT     INS FUNDS',
#  '/BOLT/EVESP',
#  '/BOLT/DAR',
 'PayU*bolt.eu',
#  'BOLT.EU /B/',
 'Taxify ..',
 'www.taxify.eu',
 'TAXIFY SOUTH AFRICA',
#  '/BOLT/MAT',
 'taxify.eu',
#  '/BOLT/TRI'
        ]
avoidlist=[
]

Construct the regex string here:

In [23]:
aliases_copy=aliases.copy()
for a in aliases_copy:
    aliases.remove(a)
    a=a.replace('\\','')
    aliases.append(a)

In [20]:
avoidlist_copy=avoidlist.copy()
for a in avoidlist_copy:
    avoidlist.remove(a)
    a=a.replace('/','')
    avoidlist.append(a)

In [29]:
aliases

['TAXIFYGLOBA_',
 'PAYU*TAXIFY.EU',
 '. sz. bolt -',
 'PAYPAELLE BOLT',
 'IKH*Taxify',
 'TAXIFY SOUTH AFRICA (P',
 'TAXIFY ..',
 'TAXIFY SOUTH AFRICA',
 'Lesedi Taxify',
 'Bolt',
 'POS BOLT -   INS FUNDS',
 'Taxify -',
 'Bolt -  -',
 'LIDL . sz. bolt',
 'PAYPAL *BOLT T',
 'TAXIFY _',
 'TAXIFY SOUTH A',
 'Taxify',
 'www.taxify.co.z',
 'TAXIFY_PAYU',
 'bolt.eu',
 'Taxify _',
 'Taxify --',
 'TAXIFY MAHLANG',
 'BOLT.EU',
 'TAXIFYGLOBA.. :',
 'POS BOLT     INS FUNDS',
 '/BOLT/EVESP',
 '/BOLT/DAR',
 'PayU*bolt.eu',
 'Taxify ..',
 'www.taxify.eu',
 'TAXIFY SOUTH AFRICA',
 'taxify.eu']

In [32]:
regex_str=mongo_regex_searchstring(aliases=aliases,avoidlist=avoidlist)
regex_str_compiled = re.compile(regex_str, re.IGNORECASE)
print(regex_str, regex_str_compiled)

Construct regex string ...
aliases =  ['TAXIFYGLOBA_', 'PAYU*TAXIFY.EU', '. sz. bolt -', 'PAYPAELLE BOLT', 'IKH*Taxify', 'TAXIFY SOUTH AFRICA', 'TAXIFY ..', 'TAXIFY SOUTH AFRICA', 'Lesedi Taxify', 'Bolt', 'POS BOLT -   INS FUNDS', 'Taxify -', 'Bolt -  -', 'LIDL . sz. bolt', 'PAYPAL *BOLT T', 'TAXIFY _', 'TAXIFY SOUTH A', 'Taxify', 'www.taxify.co.z', 'TAXIFY_PAYU', 'bolt.eu', 'Taxify _', 'Taxify --', 'TAXIFY MAHLANG', 'BOLT.EU', 'TAXIFYGLOBA.. :', 'POS BOLT     INS FUNDS', '/BOLT/EVESP', '/BOLT/DAR', 'PayU*bolt.eu', 'Taxify ..', 'www.taxify.eu', 'TAXIFY SOUTH AFRICA', 'taxify.eu']
regex_alias_str =  TAXIFYGLOBA_|PAYU*TAXIFY.EU|. sz. bolt -|PAYPAELLE BOLT|IKH*Taxify|TAXIFY SOUTH AFRICA|TAXIFY ..|TAXIFY SOUTH AFRICA|Lesedi Taxify|Bolt|POS BOLT -   INS FUNDS|Taxify -|Bolt -  -|LIDL . sz. bolt|PAYPAL *BOLT T|TAXIFY _|TAXIFY SOUTH A|Taxify|www.taxify.co.z|TAXIFY_PAYU|bolt.eu|Taxify _|Taxify --|TAXIFY MAHLANG|BOLT.EU|TAXIFYGLOBA.. :|POS BOLT     INS FUNDS|/BOLT/EVESP|/BOLT/DAR|PayU*bolt.eu|Ta

Check if the names exist, this may take a minute or two!

### The following snippet search franchisenames for the regular expression string defined by regex_str (in the cell above).  It searches in the collection defined by 'period' and ONLY searched for entries between dates as defined by start_date and end_date.
``` 
It gives output that serves as a guide.  It may take a few minutes to run depending how many transactions are found between the two dates.  The list of franchises that the following snipper returns is an UNIQUE list.  Use the entries to decide how the regexstr should be changed to improve accuracy.
```

In [33]:
regex_str

'(?m)^.*TAXIFYGLOBA_|PAYU*TAXIFY.EU|. sz. bolt -|PAYPAELLE BOLT|IKH*Taxify|TAXIFY SOUTH AFRICA|TAXIFY ..|TAXIFY SOUTH AFRICA|Lesedi Taxify|Bolt|POS BOLT -   INS FUNDS|Taxify -|Bolt -  -|LIDL . sz. bolt|PAYPAL *BOLT T|TAXIFY _|TAXIFY SOUTH A|Taxify|www.taxify.co.z|TAXIFY_PAYU|bolt.eu|Taxify _|Taxify --|TAXIFY MAHLANG|BOLT.EU|TAXIFYGLOBA.. :|POS BOLT     INS FUNDS|/BOLT/EVESP|/BOLT/DAR|PayU*bolt.eu|Taxify ..|www.taxify.eu|TAXIFY SOUTH AFRICA|taxify.eu.*$'

In [34]:
collection=client.behaviourdb['clientswipes_{period}'.format(period=period)]
find_dict= {
    'TransactionAmount':{'$lt':0},
#     'TransactionDate':datetime(2019,7,19),
    'franchisename': 
        {
            '$regex': re.compile(regex_str, re.IGNORECASE)
        },
    'companyname': 
        {
            '$exists': False
        }    
    }


df=pd.DataFrame(list(collection.find(find_dict)))

df.shape
franchisenamelist = list(set(df.franchisename))
franchisenamelist.sort(reverse = True)
franchisenamelist

['XOX BOLT AND NUT',
 'XOX BOLT AND N',
 'WORCESTER BOLT',
 'WITBANK BOLT & NUT S',
 'WITBANK BOLT &',
 'WHOLESALE BOLT AND HAR',
 'WHOLESALE BOLT',
 'WESTRAND BOLT AND TOOL',
 'WESTRAND BOLT A',
 'WESTRAND BOLT',
 'VER-BOLT (PTY) LTD',
 'VER-BOLT (PT',
 'VANDERBOLT COFFEE COMP',
 'VANDERBOLT COF',
 'UNIVERSAL BOLTS AND NU',
 'UNIVERSAL BOLT',
 'TRIPLE BOLT',
 'TRIANGLE BOLT & NUT',
 'TEMSIK BOLT AND NUTS WHOL',
 'TEMSIK BOLT AND NUTS W',
 'TEMSIK BOLT AN',
 'TEENGS BOLT AND NUT',
 'TEENGS BOLT AND NU',
 'TEENGS BOLT AN',
 'STUDBOLT DISTRIBUTOR',
 'STUDBOLT DISTR',
 'STEEL & BOLT CENTRE CC',
 'STEEL & BOLT CENTRE',
 'STEEL & BOLT C',
 'STEEL & BOLT',
 'SPEEDY NUT AND BOLT',
 'SOUTHERN CAPE BOLT (PTY',
 'SOUTHERN CAPE BOLT (PT',
 'SOUTHERN CAPE BOLT (',
 'SILVERTON BOLT',
 'RUSTENBURG BOLT & TOOL',
 'RUSTENBURG BOLT & TO',
 'RENTTECH BOLTO',
 'RELIABLE BOLT',
 'RAND BOLT SUPPLIES',
 'RAND BOLT SUPP',
 'RAND BOLT SU',
 'Q BOLT',
 'PRESIDENT BOLT',
 'PREMIER BEARING BOLT A',
 'PREMIER BEA

In [ ]:
df[df.franchisename=='/BOLTON/C'].TransactionDate

#### Look at the amount of transactions found.  Shape returns (rows, columns).

In [ ]:
franchisenamelist.sort()
franchisenamelist

In [61]:
franchise_df=pd.DataFrame()

In [64]:
franchisename_df=df[['franchisename','TransactionAmount']].groupby(['franchisename']).sum()
franchisename_df.rename(columns={'TransactionAmount':'amount'}, inplace=True)
franchisecount_df=df[['franchisename','TransactionAmount']].groupby(['franchisename']).count()
franchisecount_df.rename(columns={'TransactionAmount':'count'}, inplace =True)
franchise_df=franchise_df.append(franchisename_df.merge(franchisecount_df, left_index=True, right_index=True))

In [65]:
franchise_df

,amount,count
franchisename,,
/BOLTMAN/,-10348.63,8
/BOLTON,-1509.22,1
/BOLTON/C,-28557.99,5
/BOLTON/G,-1097.61,1
/BOLTON/M BS,-345.00,2
A J BOLTS &,-27238.44,80
A J BOLTS & NUTS,-3568.80,7
ACTION BOLT- J,-83.26,1
ACTION BOLT- JOHANNE,-6155.65,3


In [40]:
franchisename_df.reset_index(inplace=True)
# franchisename_df.rename(columns={'TransactionAmount':'amount'})
franchisecount_df.reset_index(inplace=True)
# franchisecount_df.rename(columns={'TransactionAmount':'count'})
franchise_df=franchisename_df.merge(franchisecount_df)

In [42]:
d.reset_index(inplace=True)

In [43]:
d.rename(columns={'TransactionAmount':'count'})

,franchisename,TransactionAmount
0,/BOLTMAN/,8
1,/BOLTON,1
2,/BOLTON/C,5
3,/BOLTON/G,1
4,/BOLTON/M BS,2
5,A J BOLTS &,80
6,A J BOLTS & NUTS,7
7,ACTION BOLT- J,1
8,ACTION BOLT- JOHANNE,3
9,ACTION BOLT-EA,16
